In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/Utils")


In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [3]:
from funs import *
from cols_list import cols_enem

## Carregando dataset filtrado

In [5]:
enem_trienio = load_trienio_enem(cols_enem)

Tamanho do dataset pré-filtragem:  5095171
Tamanho do dataset pós-filtragem:  1604459
Taxa de filtragem:  0.6851020309230054
Tamanho do dataset pré-filtragem:  5783109
Tamanho do dataset pós-filtragem:  1254237
Taxa de filtragem:  0.7831206363220891
Tamanho do dataset pré-filtragem:  2685053
Tamanho do dataset pós-filtragem:  950911
Taxa de filtragem:  0.6458501936460844


In [6]:
enem_trienio['NT_MEDIA'] = enem_trienio.apply(lambda row: get_medium_score(row), axis=1)

/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/dask/dataframe/core.py:5408: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=(None, 'float64'))

  warnings.warn(meta_warning(meta))


In [8]:
enem_trienio['LOG_NT_MEDIA'] = np.log(enem_trienio['NT_MEDIA'])

/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
enem_trienio = enem_trienio.compute()

In [9]:
# Presença de internet x Nota Média
mod = smf.quantreg("LOG_NT_MEDIA ~ Q025", enem_trienio)
res = mod.fit(q=0.5)
print(res.summary())

/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/numpy/core/_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


                         QuantReg Regression Results                          
Dep. Variable:           LOG_NT_MEDIA   Pseudo R-squared:                  nan
Model:                       QuantReg   Bandwidth:                         nan
Method:                 Least Squares   Sparsity:                          nan
Date:                Mon, 08 Aug 2022   No. Observations:              2832618
Time:                        16:24:15   Df Residuals:                  2832616
                                        Df Model:                            1
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept         nan        nan        nan        nan         nan         nan
Q025[T.B]         nan        nan        nan        nan         nan         nan
